In [28]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sympy
from sympy import isprime, primerange
from tqdm import tqdm
from copy import copy

%matplotlib inline

## Prepare input

In [2]:
cities = pd.read_csv('../input/cities.csv', index_col=['CityId'])

In [4]:
cities1k = cities * 1000

In [3]:
class Tour(list):  # list with trailing 0
    penalized = ~cities.index.isin(sympy.primerange(0, len(cities)))

    @staticmethod
    def from_file(filename):  # from linkern's output or csv
        seq = [int(x) for x in open(filename).read().split()[1:]]
        return Tour(seq if seq[-1] == 0 else (seq + [0]))

    def score(self):
        df = cities.reindex(self)
        dist = np.hypot(df.X.diff(-1), df.Y.diff(-1))
        penalty = 0.1 * dist[9::10] * self.penalized[self[9::10]]
        _score = dist.sum() + penalty.sum()
        _score = np.round(_score,2)

        return _score

    def to_csv(self, filename):
        pd.DataFrame({'Path': self}).to_csv(filename, index=False)

In [4]:
tour = Tour.from_file('../../../Downloads/LKH-2.0.9/data/linkern.tour')
#tour.to_csv('pool/{}.csv'.format(tour.score()))
print(tour.score())

1517113.59


In [5]:
from math import hypot
class Node:
    """
    represents a node in a TSP tour
    """
    def __init__(self, num, coords):
        self.num = num # start position in a route's order
        self.x = coords[0]   # x coordinate
        self.y = coords[1]   # y coordinate

    def __str__(self):
        """
        returns the string representation of a Node
        """
        return self.num

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def euclidean_dist(self, other):
        """
        returns the Euclidean distance between this Node and other Node
        other - other node
        """
        dx = self.x - other.x
        dy = self.y - other.y
        return hypot(dx, dy)

In [6]:
PATH = []
for i in range(len(tour)):
    PATH.append(Node(tour[i], cities.values[tour[i]]))

In [7]:
def sscore(p):
    distance = 0.0
    for i in range(len(p)-1):
        d = p[i].euclidean_dist(p[i+1])
        if i%10 == 0 and not isprime(p[i-1].num): d *= 1.1 
        distance += d
    return distance

In [8]:
sscore(PATH)

1517126.8455356655

197769

# Newly improved GSX (GSX-2).

In [209]:
parent1 = list(range(1,42))
parent1 = np.random.permutation(parent1)
parent1 = np.insert(parent1, 0, 0)
parent1 = np.insert(parent1, len(parent1), 0)
parent1

array([ 0, 41, 34,  4, 29,  6, 22, 12,  3, 28, 17,  5, 18, 14, 25, 15, 38,
       35,  8, 26, 27, 24,  7, 11, 31, 40, 19, 21, 20, 30,  9, 10,  1, 23,
        2, 16, 37, 36, 39, 32, 13, 33,  0])

In [210]:
parent2 = list(range(1,42))
parent2 = np.random.permutation(parent2)
parent2 = np.insert(parent2, 0, 0)
parent2 = np.insert(parent2, len(parent2), 0)
parent2

array([ 0, 21, 23,  1, 13, 34,  2,  6, 40,  8, 41, 26, 14, 30, 11, 16,  9,
       38,  5, 29, 10,  3, 22, 25, 36, 31, 20, 39, 35, 18, 32, 17, 37,  4,
       15, 28,  7, 24, 33, 19, 12, 27,  0])

### Step 1 : select random city 

In [211]:
rnd_city = np.random.randint(low=1, high=43)
rnd_city

29

### Step 2: alternately add cities from parents, each for one direction, until either a city that has already been included in the offspring is met or exchangeable subtours are found

In [212]:


offspring = [rnd_city]
rnd_city_in_parent1 = np.argwhere(parent1 == rnd_city)[0][0]
rnd_city_in_parent2 = np.argwhere(parent2 == rnd_city)[0][0]
city_included = [rnd_city]
STOP = False
idx1 = rnd_city_in_parent1 + 1
idx2 = rnd_city_in_parent2 - 1
while not STOP:
    if idx1 < len(parent1)-1: ## -1 for the 0 
        if parent1[idx1] in city_included:
            STOP = True
            break
        offspring.append(parent1[idx1])
        city_included.append(parent1[idx1])
        idx1+=1
    else:
        break
 
    if idx2 > 0: ## > strict for the 0
        if parent2[idx2] in city_included:
            STOP = True
            break
        offspring.insert( 0,parent2[idx2])
        city_included.append(parent2[idx2])
        idx2-=1

In [213]:
offspring

[30, 11, 16, 9, 38, 5, 29, 6, 22, 12, 3, 28, 17]

### Step 3: fill in the remaining cities at the order of parent 2

In [214]:
for i in range(1, len(parent2)-1): ##-1 for the zero
    if parent2[i] not in city_included:
        offspring.append(parent2[i])

In [215]:
offspring.append(0)
offspring.insert(0, 0)

In [221]:
print(offspring)

[0, 30, 11, 16, 9, 38, 5, 29, 6, 22, 12, 3, 28, 17, 21, 23, 1, 13, 34, 2, 40, 8, 41, 26, 14, 10, 25, 36, 31, 20, 39, 35, 18, 32, 37, 4, 15, 7, 24, 33, 19, 27, 0]


In [217]:
print( Tour(parent1).score(), Tour(parent2).score(), Tour(offspring).score())

93259.18 97120.8 88758.16


In [222]:
assert len(offspring)==len(parent1)==len(parent2)
assert offspring[0]==offspring[-1]==0
assert len(set(offspring))==len(offspring)-1

 ## Double-bridge move mutation operator

In [219]:
i, ii, j, jj, k, kk, l, ll = 3, 4, 9, 10, 28, 29, 35, 36

In [220]:
new_route = offspring[0:i+1]

new_route.append(offspring[kk])
new_route.extend(offspring[kk+1:l+1])
new_route.append(offspring[jj])
new_route.extend(offspring[jj+1:k+1])
new_route.append(offspring[ii])
new_route.extend(offspring[ii+1:j+1])
new_route.append(offspring[ll])
new_route.extend(offspring[ll+1:])

print(new_route)
print("+++++++++++++")
print(offspring)


assert len(new_route)==len(offspring)
assert new_route[0]==new_route[-1]==0
assert len(set(new_route))==len(new_route)-1
print( Tour(offspring).score(), Tour(new_route).score())




[0, 30, 11, 16, 20, 39, 35, 18, 32, 37, 4, 12, 3, 28, 17, 21, 23, 1, 13, 34, 2, 40, 8, 41, 26, 14, 10, 25, 36, 31, 9, 38, 5, 29, 6, 22, 15, 7, 24, 33, 19, 27, 0]
+++++++++++++
[0, 30, 11, 16, 9, 38, 5, 29, 6, 22, 12, 3, 28, 17, 21, 23, 1, 13, 34, 2, 40, 8, 41, 26, 14, 10, 25, 36, 31, 20, 39, 35, 18, 32, 37, 4, 15, 7, 24, 33, 19, 27, 0]
88758.16 88826.27


In [206]:
for  i in offspring:
    if i not in new_route:
        print (i)

3
37
2


In [335]:
def swap_2opt(route, i, k):
    """
    swaps the endpoints of two edges by reversing a section of nodes, 
    ideally to eliminate crossovers
    returns the new route created with a the 2-opt swap
    route - route to apply 2-opt
    i - start index of the portion of the route to be reversed
    k - index of last node in portion of route to be reversed
    pre: 0 <= i < (len(route) - 1) and i < k < len(route)
    post: length of the new route must match length of the given route 
    """
    assert i >= 0 and i < (len(route) - 1)
    assert k > i and k < len(route)
    new_route = route[0:i]
    new_route.extend(reversed(route[i:k + 1]))
#    reversed(route[i:k + 1])
    new_route.extend(route[k+1:])
    assert len(new_route)==len(route)
    assert new_route[0]==new_route[-1]==0
    assert len(set(new_route))==len(new_route)-1
    return Tour(new_route)

In [143]:
sscore(swap_2opt(PATH, 3,800))

1518858.3670700453

In [339]:
def run_2opt(route):
    """
    improves an existing route using the 2-opt swap until no improved route is found
    best path found will differ depending of the start node of the list of nodes
        representing the input tour
    returns the best path found
    route - route to improve
    """
    improvement = True
    best_route = route
    best_distance = route.score()
    while improvement: 
        improvement = False
        for i in range(1,len(best_route) - 1):
            for k in range(i+1, len(route)-1):
                new_route = swap_2opt(best_route, i, k)
                new_distance = new_route.score()
                if new_distance < best_distance:
                    #print(new_distance)
                    best_distance = new_distance
                    best_route = new_route
                    improvement = True
                    break #improvement found, return to the top of the while loop
                if improvement:
                    break
    assert len(best_route)==len(route)
    assert best_route[0]==best_route[-1]==0
    assert len(set(best_route))==len(best_route)-1
    
    return best_route

In [340]:
run_2opt(Tour(offspring)) ## 2-opt seul 20513.75

KeyboardInterrupt: 

In [351]:
population = []
popSize = 10
nGen = 100
SP = 2.0
mut_rate = .4
for i in range(popSize):
    ind = generate_ind()
    ind.pos = i
    population.append(ind)

In [352]:
population = sorted(population, key=lambda x: x.fitness, reverse=True)
for i in range(len(population)):
    population[i].rank = get_rank(popSize, SP, i)

In [353]:
for i in range(nGen):
    population = sorted(population, key=lambda x: x.fitness, reverse=True)
    for j in range(len(population)):
        population[j].rank = get_rank(popSize, SP, j)
    population = sorted(population, key=lambda x: x.rank, reverse=False)
        
    print("Gen {} - Best: {}".format(i, population[-1].fitness))
    
    if np.random.random() < mut_rate:
        parent = population[-1] 
        child = mutate(parent)
    else:
        p1 = population[-1] 
        p2 = population[-2] 
        child = crossover(p1, p2)

    child.path = run_2opt(Tour(child.path))
    child.fitness = Tour(child.path).score()
    
    if child.fitness < population[0].fitness:
        population[0] = child
    

Gen 0 - Best: 87337.94
Gen 1 - Best: 21316.16
Gen 2 - Best: 20754.7
Gen 3 - Best: 20754.7
Gen 4 - Best: 20754.7
Gen 5 - Best: 20754.7
Gen 6 - Best: 20754.7
Gen 7 - Best: 20546.52
Gen 8 - Best: 20546.52
Gen 9 - Best: 20546.52
Gen 10 - Best: 20546.52
Gen 11 - Best: 20546.52
Gen 12 - Best: 20546.52
Gen 13 - Best: 20546.52
Gen 14 - Best: 20546.52
Gen 15 - Best: 20546.52
Gen 16 - Best: 20546.52
Gen 17 - Best: 20546.52
Gen 18 - Best: 20546.52
Gen 19 - Best: 20546.52
Gen 20 - Best: 20546.52
Gen 21 - Best: 20546.52
Gen 22 - Best: 20546.52
Gen 23 - Best: 20546.52
Gen 24 - Best: 20546.52
Gen 25 - Best: 20546.52
Gen 26 - Best: 20546.52
Gen 27 - Best: 20546.52
Gen 28 - Best: 20546.52
Gen 29 - Best: 20546.52
Gen 30 - Best: 20546.52
Gen 31 - Best: 20546.52
Gen 32 - Best: 20546.52
Gen 33 - Best: 20546.52
Gen 34 - Best: 20546.52
Gen 35 - Best: 20546.52
Gen 36 - Best: 20546.52
Gen 37 - Best: 20546.52
Gen 38 - Best: 20546.52
Gen 39 - Best: 20546.52
Gen 40 - Best: 20546.52
Gen 41 - Best: 20546.52
Gen 42 

In [ ]:
Best: 20546.52

In [ ]:
class Individual():
    def __init__(self, path):
        self.path = path
        self.fitness = Tour(path).score()
        self.ranking = 0


In [265]:
def get_rank(pop_size, selection_pressure, position):
    rank = 2 - selection_pressure + 2*(selection_pressure-1)*((position-1)/(pop_size-1))
    return rank

In [329]:
def generate_ind():
    indiv = list(range(1,42))
    indiv = np.random.permutation(indiv)
    indiv = np.insert(indiv, 0, 0)
    indiv = np.insert(indiv, len(indiv), 0)
    indiv = list(indiv)
    
    assert indiv[0]==indiv[-1]==0
    assert len(set(indiv))==len(indiv)-1
    
    return Individual(indiv)

In [344]:
def mutate(indiv):
#    i, ii, j, jj, k, kk, l, ll = 3, 4, 9, 10, 28, 29, 35, 36
    
    i = np.random.randint(1, int(0.25*len(indiv.path)))
    ii = i+1
    
    j = np.random.randint(int(0.25*len(indiv.path))+1, int(0.50*len(indiv.path)))
    jj = j+1
    
    k = np.random.randint(int(0.50*len(indiv.path))+1, int(0.75*len(indiv.path)))
    kk = k+1
    
    l = np.random.randint(int(0.75*len(indiv.path))+1, len(indiv.path)-2)
    ll = l+1
    
    new_genome = indiv.path[0:i+1]

    new_genome.append(indiv.path[kk])
    new_genome.extend(indiv.path[kk+1:l+1])
    new_genome.append(indiv.path[jj])
    new_genome.extend(indiv.path[jj+1:k+1])
    new_genome.append(indiv.path[ii])
    new_genome.extend(indiv.path[ii+1:j+1])
    new_genome.append(indiv.path[ll])
    new_genome.extend(indiv.path[ll+1:])


    assert len(new_genome)==len(indiv.path)
    assert new_genome[0]==new_genome[-1]==0
    assert len(set(new_genome))==len(new_genome)-1
    
    return Individual(new_genome)

In [328]:
def crossover(p1, p2):
    
    parent1, parent2 = p1.path, p2.path
    rnd_city = np.random.randint(low=1, high=42)
    offspring = [rnd_city]
    rnd_city_in_parent1 = parent1.index(rnd_city)
    rnd_city_in_parent2 = parent2.index(rnd_city)
    city_included = [rnd_city]
    STOP = False
    idx1 = rnd_city_in_parent1 + 1
    idx2 = rnd_city_in_parent2 - 1
    while not STOP:
        if idx1 < len(parent1)-1: ## -1 for the 0 
            if parent1[idx1] in city_included:
                STOP = True
                break
            offspring.append(parent1[idx1])
            city_included.append(parent1[idx1])
            idx1+=1
        else:
            break

        if idx2 > 0: ## > strict for the 0
            if parent2[idx2] in city_included:
                STOP = True
                break
            offspring.insert( 0,parent2[idx2])
            city_included.append(parent2[idx2])
            idx2-=1
    for i in range(1, len(parent2)-1): ##-1 for the zero
        if parent2[i] not in city_included:
            offspring.append(parent2[i])
    offspring.append(0)
    offspring.insert(0, 0)
    
    assert len(offspring)==len(p1.path)
    assert offspring[0]==offspring[-1]==0
    assert len(set(offspring))==len(offspring)-1
    
    return Individual(offspring)

In [294]:
parent = population[-1] 
child = mutate(parent)
child
crossover(parent, child)

In [152]:
def reverse_segment_if_better(tour, i, j, k):
    "If reversing tour[i:j] would make the tour shorter, then do it."
    # Given tour [...A-B...C-D...E-F...]
    A, B, C, D, E, F = tour[i-1], tour[i], tour[j-1], tour[j], tour[k-1], tour[k % len(tour)]
    d0 = A.euclidean_dist(B) + C.euclidean_dist(D) + E.euclidean_dist(F)
    d1 = A.euclidean_dist(C) + B.euclidean_dist(D) + E.euclidean_dist(F)
    d2 = A.euclidean_dist(B) + C.euclidean_dist(E) + D.euclidean_dist(F)
    d3 = A.euclidean_dist(D) + E.euclidean_dist(B) + C.euclidean_dist(F)
    d4 = F.euclidean_dist(B) + C.euclidean_dist(D) + E.euclidean_dist(A)

    if d0 > d1:
      tour[i:j] = reversed(tour[i:j])
      return -d0 + d1
    elif d0 > d2:
      tour[j:k] = reversed(tour[j:k])
      return -d0 + d2
    elif d0 > d4:
      tour[i:k] = reversed(tour[i:k])
      return -d0 + d4
    elif d0 > d3:
      tmp = tour[j:k], tour[i:j]
      tour[i:k] = tmp
      return -d0 + d3
    return 0

def three_opt(tour):
    "Iterative improvement based on 3 exchange."
    while True:
        delta = 0
        print(sscore(tour))
        for (a,b,c) in all_segments(len(tour)):
            delta += reverse_segment_if_better(tour, a, b, c)
        if delta >= 0:
            break
        print(sscore(tour))
    return tour

def all_segments(N):
    "Generate all segments combinations"
    return ((i, j, k)
        for i in range(N)
        for j in range(i+2, N)
        for k in range(j+2, N+(i>0)))

In [156]:
three_opt(tour)

AttributeError: 'int' object has no attribute 'euclidean_dist'

In [75]:

def print_results(route, filename, time, startnode):
    """
    prints the nodes in the final route and route information
    route - route to print
    filename - name of the original input filename
    time - time to run 2opt
    startnode - start node of original tour if randomized
    """
    for node in route:
        print(node)
    print(-1)
    print("Original input file : " + filename)
    print("Dimension : " + str(len(route)))
    if startnode:
        print("Randomized start node : " + str(startnode))
    print("Total Distance : " + str(route.score()))
    print("Time to run 2opt : %.2f seconds" % time)

In [51]:
route = run_2opt(tour)

  0%|          | 0/17803 [00:00<?, ?it/s]

40096995.92
40094304.0
40094273.46
40093798.46
40093366.32
40092276.55



  0%|          | 0/17803 [00:00<?, ?it/s]

40092185.11
40091972.84
40091488.84
40090422.44
40087992.76
40087972.15
40087634.29



  0%|          | 0/17803 [00:00<?, ?it/s]

...


  0%|          | 9/17803 [00:00<04:46, 62.11it/s]

39888160.34


  0%|          | 0/17803 [00:00<?, ?it/s]


39887567.33
39887112.8


KeyboardInterrupt: 